# 1. Parameters

In [ ]:
from pathlib import Path

# Defaults
cases_dir = 'cases/full'
metadata_file = 'metadata.tsv'
iterations = 3
mincov = 10
ncores = 4

reference_files = {
    '0810PADBR-1': Path('data/reference/campylobacter/GCA_001879185.2_ASM187918v2_genomic.gbk.gz'),
    '1405WAEXK-1': Path('data/reference/ecoli/GCF_000703365.1_Ec2011C-3609_genomic.gbk.gz'),
    '1408MLGX6-3WGS': Path('data/reference/listeria/GCF_001047715.2_ASM104771v2_genomic.gbk.gz'),
    '1203NYJAP-1 - Tuna Scrape Outbreak': Path('data/reference/salmonella/GCF_000439415.1_ASM43941v1_genomic.gbk.gz'),
}
    
build_tree = True
sample_batch_size=10

In [ ]:
from pathlib import Path
from shutil import rmtree
from os import makedirs
import imp
fp, pathname, description = imp.find_module('gdi_benchmark', ['../../lib'])
gdi_benchmark = imp.load_module('gdi_benchmark', fp, pathname, description)

cases_dir_path = Path(cases_dir)
index_path = cases_dir_path / 'index'

if cases_dir_path.exists():
    rmtree(cases_dir_path)
    
if not cases_dir_path.exists():
    makedirs(cases_dir_path)

case_name = str(cases_dir_path.name)

benchmark_path = cases_dir_path / 'index-info.tsv'

output_trees = {x: cases_dir_path / f'{x}_tree.tre'.replace(' ', '_') for x in reference_files}
output_trees

# 2. Create subset inputs

In [ ]:
import pandas as pd
from pathlib import Path
from typing import Dict

metadata_df = pd.read_csv(metadata_file, sep='\t')

def write_subset_input(metadata_df: pd.DataFrame, dataset_name: str) -> Path:
    all_input_total = len(metadata_df)
    cases_input = cases_dir_path / f'{dataset_name}_input-files-case.tsv'.replace(' ', '_')

    input_df = metadata_df.copy().loc[metadata_df['dataSetName'] == dataset_name]
    input_df['Sample'] = input_df['strain']
    input_df['Assemblies'] = pd.NA
    input_df['Reads1'] = input_df['Sample'].apply(lambda x: str((Path('data') / 'fastq' / (x + '_1.fastq.gz')).absolute()))
    input_df['Reads2'] = input_df['Sample'].apply(lambda x: str((Path('data') / 'fastq' / (x + '_2.fastq.gz')).absolute()))
    input_df = input_df[['Sample', 'Assemblies', 'Reads1', 'Reads2']]

    input_df.to_csv(cases_input, sep='\t', index=False)

    subset_input_total = len(input_df)

    print(f'Wrote dataset={dataset_name} consisting of {subset_input_total}/{all_input_total} samples to {cases_input}')
    
    return cases_input

cases_inputs = {x: write_subset_input(metadata_df, dataset_name=x) for x in reference_files}
cases_inputs

# 2. Index genomes

In [ ]:
!gdi --version

## 2.1. Index reads

In [ ]:
import time

start = time.time()
benchmarker = gdi_benchmark.IndexBenchmarkerMultiple(index_path=index_path, input_files_files=cases_inputs,
                                             reference_files=reference_files, mincov=mincov,
                                             build_tree=build_tree,
                                             ncores=ncores,
                                             sample_batch_size=sample_batch_size)
benchmark_df = benchmarker.benchmark(iterations=iterations)
end = time.time()
print(f'***Finished benchmarking, took {(end - start)/60:0.2f} minutes***')

In [ ]:
benchmark_df

In [ ]:
benchmark_df.to_csv(benchmark_path, sep='\t', index=False)

# 3. Export trees

In [ ]:
if build_tree:
    # Using variables with shell commands in Jupyter isn't working for me so I have to avoid using them
    !gdi --project-dir cases/full/index export tree GCA_001879185.2_ASM187918v2_genomic > cases/full/0810PADBR-1_tree.tre
    !gdi --project-dir cases/full/index export tree GCF_000703365.1_Ec2011C-3609_genomic > cases/full/1405WAEXK-1_tree.tre
    !gdi --project-dir cases/full/index export tree GCF_001047715.2_ASM104771v2_genomic > cases/full/1408MLGX6-3WGS_tree.tre
    !gdi --project-dir cases/full/index export tree GCF_000439415.1_ASM43941v1_genomic > cases/full/1203NYJAP-1_-_Tuna_Scrape_Outbreak_tree.tre
else:
    print(f'build_tree={build_tree} so no trees to export')